# Train summary for a single episode

This notebook presents plots related to a single episode. It can display plots about an episode from two sources:

1. **Training iterations**: each iteration plays the same number of episodes, but the episodes are different as they use randomly generated seeds.

2. **Evaluation iterations**: each iteration also plays the same number of episodes, but they use the same list of seeds across iterations. This is to evaluate the agents over time.

## Experiment loading

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib widget
import base

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import dfaas_env
import dfaas_utils

In [ ]:
# Base directory where the experiments are located..
prefix_dir = Path("/home/emanuele/marl-dfaas/results/")

# Experiment directory.
exp_dir = prefix_dir / "DF_20250707_165017_PPO_test"

# Which training statistics to show: from training iterations or evaluation
# iterations.
mode = "eval"  # Or "eval"

if mode == "train":
    # The result file is a JSON with each entry separated by newline.
    raw_exp_data = dfaas_utils.parse_result_file(exp_dir / "result.json")
elif mode == "eval":
    # The evaluation file is instead a normal JSON.
    raw_exp_data = dfaas_utils.json_to_dict(exp_dir / "evaluation.json")
else:
    raise ValueError(f"Invalid {mode = }")

# Reference environment.
env = base.get_env(exp_dir)

In [ ]:
print(f"Experiment prefix dir: {prefix_dir.as_posix()!r}")
print(f"Experiment name:       {exp_dir.name!r}")
print(f"Agents:                {env.agents} ({len(env.agents)})")
print(f"Mode:                  {mode}")
if mode == "train":
    print(f"Iterations:            {len(raw_exp_data)}")
else:
    print(f"Evaluations:           {len(raw_exp_data)}")

In [ ]:
# Which iteration (training or evaluation) to select?
iteration_idx = 2

# Which episode from the single iteration to select?
episode_idx = 0

assert 0 <= iteration_idx <= len(raw_exp_data) - 1, "iteration_idx must be a valid iteration index!"

assert (
    0 <= episode_idx < raw_exp_data[iteration_idx]["env_runners"]["num_episodes"]
), "episode_idx must be a valid episode index!"

In [ ]:
print(f"Selected iteration:    {iteration_idx}")
print(f"Selected episode:      {episode_idx}")

## Reward

In [ ]:
def make_reward_plot(raw_exp_data, env, iteration_idx, episode_idx):
    for agent in ["all"] + env.agents:
        plt.close(fig=f"reward_{agent}")
        fig = plt.figure(num=f"reward_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        if agent == "all":
            reward = np.add(
                *[
                    raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["reward"][episode_idx][agent]
                    for agent in env.agents
                ]
            )
        else:
            reward = raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["reward"][episode_idx][agent]

        ax.plot(reward)
        ax.set_title(f"Reward per step ({agent = })")
        ax.set_ylabel("Reward")
        ax.set_xlabel("Step")
        # ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))

        ax.grid(axis="both")
        ax.set_axisbelow(True)


make_reward_plot(raw_exp_data, env, iteration_idx, episode_idx)

## Input rate

In [ ]:
def make_input_rate_plot(raw_exp_data, env, iteration_idx, episode_idx):
    for agent in ["all"] + env.agents:
        plt.close(fig=f"input_rate_{agent}")
        fig = plt.figure(num=f"input_rate_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        if agent == "all":
            input_rate = np.add(
                *[
                    raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["observation_input_rate"][episode_idx][
                        agent
                    ]
                    for agent in env.agents
                ]
            )
        else:
            input_rate = raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["observation_input_rate"][
                episode_idx
            ][agent]

        ax.plot(input_rate)
        ax.set_title(f"Input rate per step ({agent = })")
        ax.set_ylabel("Input rate")
        ax.set_xlabel("Step")
        ax.xaxis.set_major_locator(ticker.MultipleLocator(25))
        ax.grid(axis="both")
        ax.set_axisbelow(True)


make_input_rate_plot(raw_exp_data, env, iteration_idx, episode_idx)

## Absolute Action

In [ ]:
def get_absolute_action(raw_exp_data, env, iteration_idx, episode_idx):
    data = {}
    for agent in env.agents:
        data[agent] = {}
        data[agent]["input_rate"] = np.array(
            raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["observation_input_rate"][episode_idx][agent]
        )
        data[agent]["action_local"] = np.array(
            raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["action_local"][episode_idx][agent]
        )
        data[agent]["action_forward"] = np.array(
            raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["action_forward"][episode_idx][agent]
        )
        data[agent]["action_reject"] = np.array(
            raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["action_reject"][episode_idx][agent]
        )

        forward_reject = np.array(
            raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["forward_reject_rate"][episode_idx][agent]
        )
        local_reject = np.array(
            raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["incoming_rate_local_reject"][episode_idx][agent]
        )
        data[agent]["reject_rate"] = forward_reject + local_reject + data[agent]["action_reject"]

    data["all"] = {}
    data["all"]["input_rate"] = np.add(*[data[agent]["input_rate"] for agent in env.agents])
    data["all"]["action_local"] = np.add(*[data[agent]["action_local"] for agent in env.agents])
    data["all"]["action_forward"] = np.add(*[data[agent]["action_forward"] for agent in env.agents])
    data["all"]["action_reject"] = np.add(*[data[agent]["action_reject"] for agent in env.agents])
    data["all"]["reject_rate"] = np.add(*[data[agent]["reject_rate"] for agent in env.agents])

    return data


absolute_action = get_absolute_action(raw_exp_data, env, iteration_idx, episode_idx)

In [ ]:
def make_absolute_action_plot(absolute_action):
    for agent, agent_absolute_action in sorted(absolute_action.items()):
        plt.close(fig=f"absolute_action_{agent}")
        fig = plt.figure(num=f"absolute_action_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        steps = np.arange(len(agent_absolute_action["input_rate"]))

        ax.plot(agent_absolute_action["input_rate"], label="Input rate")
        ax.stackplot(
            steps,
            agent_absolute_action["action_local"],
            agent_absolute_action["action_forward"],
            agent_absolute_action["action_reject"],
            labels=["Action Local", "Action Forward", "Action Reject"],
            alpha=0.5,
        )
        ax.plot(agent_absolute_action["reject_rate"], label="Reject rate")

        ax.set_title(f"Action distribution per episode ({agent = })")
        ax.set_ylabel("Requests")
        ax.set_xlabel("Step")
        # Use only integer ticks and let matplotlib to choose how many ticks to show.
        # ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

        ax.legend()
        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.


make_absolute_action_plot(absolute_action)

## Action

In [ ]:
def make_action_distribution_plot(absolute_action, env, iteration_idx, episode_idx):
    for agent in ["all"] + env.agents:
        plt.close(fig=f"action_distribution_{agent}")
        fig = plt.figure(num=f"action_distribution_{agent}", layout="constrained")
        fig.canvas.header_visible = False

    action_local_tmp, action_forward_tmp, action_reject_tmp = [], [], []
    for agent in env.agents + ["all"]:
        fig = plt.figure(num=f"action_distribution_{agent}")
        ax = fig.subplots()

        steps = np.arange(env.max_steps)

        if agent == "all":  # Always executed at the end.
            # Sum all columns (one row for each agent).
            action_local = np.array(action_local_tmp).sum(axis=0)
            action_forward = np.array(action_forward_tmp).sum(axis=0)
            action_reject = np.array(action_reject_tmp).sum(axis=0)
        else:
            action_local = np.array(
                raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["action_local"][episode_idx][agent]
            )
            action_forward = np.array(
                raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["action_forward"][episode_idx][agent]
            )
            action_reject = np.array(
                raw_exp_data[iteration_idx]["env_runners"]["hist_stats"]["action_reject"][episode_idx][agent]
            )

            action_local_tmp.append(action_local)
            action_forward_tmp.append(action_forward)
            action_reject_tmp.append(action_reject)

        input_rate = action_local + action_forward + action_reject

        # Convert to ratios, but make sure to avoid division by zero (when input_rate is zero).
        action_local = np.divide(action_local, input_rate, out=np.zeros(env.max_steps), where=input_rate != 0) * 100
        action_forward = np.divide(action_forward, input_rate, out=np.zeros(env.max_steps), where=input_rate != 0) * 100
        action_reject = np.divide(action_reject, input_rate, out=np.zeros(env.max_steps), where=input_rate != 0) * 100

        ax.stackplot(
            steps, action_local, action_forward, action_reject, labels=["Local", "Forward", "Reject"], alpha=0.8
        )

        ax.set_title(f"Action distribution per step ({agent = })")
        ax.set_ylabel("Action")
        ax.set_xlabel("Step")
        ax.yaxis.set_major_formatter(ticker.PercentFormatter())
        ax.xaxis.set_major_locator(ticker.MultipleLocator(25))
        ax.legend()
        ax.grid(axis="both")
        ax.set_axisbelow(True)


episode_idx

make_action_distribution_plot(raw_exp_data, env, iteration_idx, episode_idx)

## Agent Reject Rate

These plots show the rejection rate of each agent and the cumulative rejection rate of all agents as a percentage of the total input rate for each step.

The agent rejection rate is the sum of the rejection rates by action, local unprocessed rates, and forwarded rates not processed by neighboring agents.

In [ ]:
def make_agent_reject_rate_plot(absolute_action):
    for agent, agent_absolute_action in sorted(absolute_action.items()):
        plt.close(fig=f"agent_reject_rate_{agent}")
        fig = plt.figure(num=f"agent_reject_rate_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        if agent != "all":
            reject_percentual = agent_absolute_action["reject_rate"] / agent_absolute_action["input_rate"] * 100
        else:
            # Cumulate the percentual over all agents for the "all" case.
            reject_rates, input_rates = [], []
            for cum_agent, cum_agent_action in absolute_action.items():
                if cum_agent == "all":
                    continue
                reject_rates.append(cum_agent_action["reject_rate"])
                input_rates.append(cum_agent_action["input_rate"])

            # Sum across agents per step.
            total_reject = np.sum(reject_rates, axis=0)
            total_input = np.sum(input_rates, axis=0)

            # Get cumulative rate for each step, avoiding division by zero (in this case just use 0).
            with np.errstate(divide="ignore"):
                reject_percentual = np.where(total_input > 0, total_reject / total_input * 100, 0)

        ax.plot(reject_percentual)

        ax.set_title(f"Agent reject rate¹ per step ({agent = })")
        ax.text(
            0.5,
            -0.2,
            "¹reject rate = action reject + local reject + forward reject",
            fontsize=10,
            ha="center",
            transform=ax.transAxes,
        )
        ax.set_ylabel("Reject rate")
        ax.yaxis.set_major_formatter(ticker.PercentFormatter())
        ax.set_xlabel("Step")

        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.


# Reuse absolute_action calculated before.
make_agent_reject_rate_plot(absolute_action)